In [3]:
from langchain_community.llms import Ollama

In [4]:
model = Ollama(model='Phi3')

C:\Users\sawan\AppData\Local\Temp\ipykernel_26348\2451165596.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model='Phi3')


In [5]:
model.invoke('hi')

'Hi there! How can I help you today?'

In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser

chain.invoke("one line definition of life")

'Life: A complex characteristic marked by growth, reproduction, functional activity, and the ability to respond to stimuli.'

In [8]:
from langchain_community.document_loaders import PyPDFLoader

In [9]:
loader = PyPDFLoader("ml_intro.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'ml_intro.pdf', 'page': 0}, page_content='Machine learning is a field of artificial intelligence that enables computers to learn and \nimprove from experience without being explicitly programmed. It involves the \ndevelopment of algorithms and statistical models that allow systems to perform specific \ntasks effectively without relying on rule-based programming.'),
 Document(metadata={'source': 'ml_intro.pdf', 'page': 1}, page_content='One of the key advantages of machine learning is its ability to adapt and evolve over time. \nAs machines are exposed to more data and experiences, they can identify patterns, make \npredictions, and refine their decision -making processes. This makes machine l earning \nparticularly useful in areas where traditional programming approaches may be \nineffective or impractical, such as image recognition, natural language processing, and \npredictive analytics.'),
 Document(metadata={'source': 'ml_intro.pdf', 'page': 2}, page_c

In [10]:
from langchain.prompts import PromptTemplate

template = '''
Answer the question from the given context only. If you dont know the answer, just say I dont know.
Context: {context}
Question: {question}
'''

prompt_template = PromptTemplate.from_template(template)

In [11]:
chain = prompt_template | model | parser

In [12]:
chain.invoke(
    {
        "context":"I am sawan rai. I have done PhD",
        "question":"what is my education?"
    }

)

'You completed your PhD and hold a title of "Sawan Rai." There\'s no additional specific educational information provided in the context. However, typically someone with a PhD would have at least a bachelor\' extramurals degree followed by master\'s degrees before their doctorate level studies commence.'

In [13]:
from langchain_community.embeddings import OllamaEmbeddings

In [14]:
embeddings = OllamaEmbeddings(model = "phi3")

C:\Users\sawan\AppData\Local\Temp\ipykernel_26348\562378373.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model = "phi3")


In [15]:
from langchain_community.vectorstores import DocArrayInMemorySearch

In [16]:
vector_store = DocArrayInMemorySearch.from_documents(pages, embeddings)

d:\LLM_work\chat_with_pdf\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [17]:
retriever = vector_store.as_retriever()
retriever.invoke("tell me something about machine learning in short.")

[Document(metadata={'source': 'ml_intro.pdf', 'page': 2}, page_content='Machine learning algorithms can be broadly categorized into supervised, unsupervised, \nand reinforcement learning. Supervised learning involves training models on labeled data \nto make predictions or decisions, while unsupervised learning identifies patterns  and \nstructures in unlabeled data. Reinforcement learning, on the other hand, focuses on \nlearning through trial and error, with the goal of maximizing a reward signal.'),
 Document(metadata={'source': 'ml_intro.pdf', 'page': 5}, page_content='Overall, machine learning is a powerful and versatile field that has the potential to \ntransform our world. As research and development in this area continue to advance, it is \ncrucial that we explore the ethical implications and work towards the responsible  \napplication of these technologies for the benefit of humanity.'),
 Document(metadata={'source': 'ml_intro.pdf', 'page': 3}, page_content='As machine learnin

In [18]:
from operator import itemgetter

In [21]:
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt_template
    | model
    | parser
)

chain.invoke({"question":"what is life?"})

'I\'m sorry, but I can only provide information based on the given context about machine learning algorithms from \'ml_intro.pdf\'. The question "What is life?" cannot be answered within this specific context as it falls outside its scope and pertains to philosophy or biology rather than being discussed in these documents.'

In [22]:
questions = ["what is machine learning?", "types of machine learning.", "what is life?"]

In [23]:
for question in questions:
    print(f"question: {question}")
    print(f"answer: {chain.invoke({'question':question})}")

question: what is machine learning?
answer: Machine learning, as described in document [Document(metadata={'source': 'ml_intro.pdf', 'page': 0}, page_content='Machine learning is a field of artificial intelligence that enables computers to learn and improve from experience without being explicitly programmed. It involves the development of algorithms and statistical models that allow systems to perform specific tasks effectively without relying on rule-based programming.')], is an area in artificial intelligence where machines are designed to automatically improve their performance through learning, by finding patterns or deriving rules based on past experiences with data. This field avoids explicit instructions for every task the machine needs to accomplish and instead focuses on enabling algorithms to learn from experience using statistical methods and computational models.
question: types of machine learning.
answer: Supervised, unsupervised, and reinforcement learning are all types

In [26]:
for c in chain.stream({"question":"what is machine learning?"}):
    print(c, end="")

Machine learning is a field of artificial intelligence that enables computers to learn and improve from experience without being explicitly programmed. It involves the development of algorithms and statistical models that allow systems to perform specific tasks effectively without relying on rule-based programming. Additionally, it's about developing programs that can access data, use it for machine learning algorithm training purposes, make informed decisions with high accuracy, learn from mistakes, improve over time etc., all while reducing the need for human intervention and increasing efficiency in automated processes across various industries such as healthcare, finance, transportation, entertainment etc.